In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from numpy import genfromtxt
import numpy as np


In [98]:
class MyMusicDataset(Dataset):
    def __init__(self, data_file_path, insert_random_data = True, n_classes = 26): # [0, 25]
        np_data = genfromtxt(data_file_path, dtype = np.int32)
        #print(np_data.shape)
        np_data_y = np.ones_like(np_data[:, 0:1]) # full score, input scores are discarded
        np_data_x = np_data[:, 1:] # sequence of music notes

        data_x = torch.Tensor(np_data_x) # integer class numbers in [0, 25]
        data_y = torch.Tensor(np_data_y) # scores are in {0, 1}
        
        
        if (insert_random_data):
            random_x = torch.randint(low = 0, high = n_classes, size = data_x.shape, dtype = torch.int)# [0, 25]
            random_y = torch.zeros_like(data_y) # random music is bad music
            data_x = torch.cat((data_x, random_x))
            data_y = torch.cat((data_y, random_y))
        data_x = torch.unsqueeze(data_x,1)
        
        # one hot representation
        #data_x = F.one_hot(data_x.to(torch.int64), num_classes = n_classes)
        print(data_x.shape) #(N,C,seqlen,num_classes)
        self.data_x = data_x.double()
        self.data_y = data_y.double()
        #print(data_y)
    
    def __len__(self):
        return len(self.data_y)
    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx] # seqs, labels



In [99]:
class MusicNet(nn.Module):
    def __init__(self, seq_len):
        super(MusicNet, self).__init__()         # N, C(annnels), L(ength of the sequence)
#         self.extractor = nn.Sequential(
#             nn.Flatten(start_dim = 2, end_dim = 3),#(N,C=1,seqlen*num_classes) 
#             nn.Conv1d(in_channels = 1, out_channels = 2, kernel_size = 52), 
#             nn.AvgPool1d(kernel_size = 4),
#             nn.Tanh(),
#             nn.Conv1d(in_channels = 2, out_channels = 2, kernel_size = 26), 
#             nn.Tanh(),
#             nn.AvgPool1d(kernel_size = 4),
#             nn.Conv1d(in_channels = 2, out_channels = 5, kernel_size = 26), 
#             nn.AvgPool1d(kernel_size = 4),
#             nn.Tanh(),
#             nn.Flatten(),
#             nn.Linear(25, 20),
#             nn.Tanh(),
#             nn.Linear(20, 1),
#             nn.Sigmoid()
#         )
        self.extractor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 200),
            nn.Sigmoid(),
            nn.Linear(200, 10),
            nn.Sigmoid(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        #print(x.shape)
        return self.extractor(x)

In [116]:
def train(model, train_loader, num_epochs, loss_fn, optimizer):
    for epoch in range(0, num_epochs):
        print("epoch", epoch)
        for batch, (X, y) in enumerate(train_loader):
            for rep in range(1):
                pred = model(X.float())
                loss = loss_fn(pred.float(), y.float())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                if (batch % 10 == 0):
                    print("y: ", y)
                    print("pred: ", pred)
                    print("loss: ", float(loss))

In [102]:
# test the dataloader
train_data = MyMusicDataset('./data/chopin_nocturnes_complete.txt', insert_random_data = True, n_classes = 28)
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
train_features, train_labels = next(iter(train_dataloader))
print(train_features)
print(train_labels)

torch.Size([1272, 1, 32])
tensor([[[22., 21., 24., 22., 21., 19., 21., 16., 17., 21., 17., 14., 26., 25.,
          24., 24., 22., 21., 24., 22., 21., 19., 21., 19., 19., 17., 14., 14.,
          16., 17., 19., 19.]],

        [[19., 19., 19., 21., 19., 19., 17., 19., 19., 21., 19., 19., 16., 19.,
          19., 16., 17., 17., 26., 24., 24., 21., 19., 19., 19., 19., 19., 21.,
          19., 19., 17., 19.]],

        [[22., 20., 20., 20.,  2.,  4.,  3.,  0.,  0.,  0., 20., 20., 20., 20.,
          20., 20., 20., 20.,  2.,  4.,  3.,  1.,  0.,  0.,  0.,  0., 18., 18.,
          18., 18., 17., 17.]],

        [[13., 16., 25., 27., 25., 25., 25., 25., 25., 23., 27., 25., 23.,  0.,
          27., 27., 25., 23., 23., 20., 25., 25., 23., 23., 25., 23., 27., 25.,
          23., 27., 23., 20.]],

        [[14., 27., 23., 13.,  7., 27., 14., 13., 10.,  2.,  7., 20., 25., 24.,
          27.,  4.,  3., 18.,  6.,  8.,  0., 14., 19., 15., 18., 10.,  7., 13.,
          26., 16.,  1.,  4.]],

        [

In [120]:
# create & test the network

net = MusicNet(32)
print(net(train_features.float()))

tensor([[0.4626],
        [0.4617],
        [0.4630],
        [0.4621],
        [0.4637],
        [0.4612],
        [0.4576],
        [0.4596],
        [0.4587],
        [0.4685],
        [0.4595],
        [0.4592],
        [0.4607],
        [0.4588],
        [0.4583],
        [0.4653]], grad_fn=<SigmoidBackward0>)


In [119]:
# prepare dataset
train_data = MyMusicDataset('./data/chopin_nocturnes_complete.txt', insert_random_data = True, n_classes = 28)
# test_data = MyMusicDataset('./data/test.txt')

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=2, shuffle=True)

torch.Size([1272, 1, 32])


In [121]:
# start the training process
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-4)
train(net, train_dataloader, 500, loss_fn, optimizer)

epoch 0
y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4647],
        [0.4586],
        [0.4684],
        [0.4637],
        [0.4618],
        [0.4617],
        [0.4725],
        [0.4567],
        [0.4623],
        [0.4615],
        [0.4602],
        [0.4637],
        [0.4604],
        [0.4608],
        [0.4617],
        [0.4624]], grad_fn=<SigmoidBackward0>)
loss:  0.2583090662956238
y:  tensor([[0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4556],
        [0.4630],
        [0.4633],
        [0.4628],
        [0.4628],
        [0.4609],
        [0.4516],
        [

y:  tensor([[1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4640],
        [0.4625],
        [0.4595],
        [0.4628],
        [0.4629],
        [0.4676],
        [0.4677],
        [0.4646],
        [0.4625],
        [0.4650],
        [0.4636],
        [0.4589],
        [0.4633],
        [0.4651],
        [0.4635],
        [0.4643]], grad_fn=<SigmoidBackward0>)
loss:  0.24713033437728882
y:  tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4618],
        [0.4581],
        [0.4669],
        [0.4626],
        [0.4650],
        [0.4637],
        [0.4641],
        [0.4614]

y:  tensor([[1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4645],
        [0.4648],
        [0.4641],
        [0.4676],
        [0.4682],
        [0.4642],
        [0.4629],
        [0.4650],
        [0.4632],
        [0.4657],
        [0.4619],
        [0.4675],
        [0.4630],
        [0.4651],
        [0.4699],
        [0.4645]], grad_fn=<SigmoidBackward0>)
loss:  0.24732321500778198
epoch 8
y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4652],
        [0.4651],
        [0.4656],
        [0.4648],
        [0.4641],
        [0.4648],
        [0.4620],
        

epoch 12
y:  tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.4663],
        [0.4585],
        [0.4663],
        [0.4668],
        [0.4618],
        [0.4637],
        [0.4639],
        [0.4622],
        [0.4651],
        [0.4656],
        [0.4633],
        [0.4622],
        [0.4642],
        [0.4615],
        [0.4671],
        [0.4638]], grad_fn=<SigmoidBackward0>)
loss:  0.2656538486480713
y:  tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.4641],
        [0.4666],
        [0.4633],
        [0.4698],
        [0.4619],
        [0.4622],
        [0.4640],
        

y:  tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.4640],
        [0.4668],
        [0.4666],
        [0.4625],
        [0.4674],
        [0.4598],
        [0.4665],
        [0.4614],
        [0.4589],
        [0.4610],
        [0.4657],
        [0.4675],
        [0.4646],
        [0.4699],
        [0.4706],
        [0.4652]], grad_fn=<SigmoidBackward0>)
loss:  0.23392803966999054
y:  tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4639],
        [0.4656],
        [0.4622],
        [0.4696],
        [0.4655],
        [0.4632],
        [0.4661],
        [0.4698]

...

y:  tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4970],
        [0.4994],
        [0.5018],
        [0.4950],
        [0.4990],
        [0.4957],
        [0.4976],
        [0.5019],
        [0.4910],
        [0.4980],
        [0.4925],
        [0.4959],
        [0.4943],
        [0.4988],
        [0.4964],
        [0.4904]], grad_fn=<SigmoidBackward0>)
loss:  0.24869567155838013
y:  tensor([[1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4994],
        [0.4918],
        [0.4940],
        [0.5008],
        [0.4959],
        [0.4878],
        [0.4910],
        [0.4983]

y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.5061],
        [0.4968],
        [0.4978],
        [0.5017],
        [0.4957],
        [0.4997],
        [0.5016],
        [0.4903],
        [0.5002],
        [0.4893],
        [0.4982],
        [0.4955],
        [0.4963],
        [0.4964],
        [0.4974],
        [0.4948]], grad_fn=<SigmoidBackward0>)
loss:  0.24869345128536224
y:  tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.4932],
        [0.4907],
        [0.4919],
        [0.4966],
        [0.4966],
        [0.5008],
        [0.4903],
        [0.4911]

pred:  tensor([[0.4940],
        [0.4959],
        [0.4972],
        [0.4964],
        [0.4964],
        [0.4976],
        [0.4961],
        [0.4979],
        [0.4970],
        [0.4970],
        [0.4956],
        [0.5062],
        [0.4944],
        [0.5010],
        [0.5007],
        [0.4974]], grad_fn=<SigmoidBackward0>)
loss:  0.2512814700603485
epoch 488
y:  tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4924],
        [0.4984],
        [0.4925],
        [0.4976],
        [0.4939],
        [0.4977],
        [0.4943],
        [0.4934],
        [0.5010],
        [0.4882],
        [0.5015],
        [0.5001],
        [0.4921],
        [0.4963],
        [0.4979],
        [0.4913]], grad_fn=<SigmoidBackward0>)
loss:  0.24848942458629608
y:  tensor([[0.],
        [0.],
       

y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.4986],
        [0.4923],
        [0.4962],
        [0.4976],
        [0.4933],
        [0.5026],
        [0.5007],
        [0.5030],
        [0.4991],
        [0.4928],
        [0.4941],
        [0.4916],
        [0.4898],
        [0.4959],
        [0.4977],
        [0.5023]], grad_fn=<SigmoidBackward0>)
loss:  0.24977900087833405
y:  tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.4992],
        [0.5001],
        [0.4972],
        [0.5053],
        [0.4958],
        [0.4977],
        [0.4946],
        [0.4972]

y:  tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.]], dtype=torch.float64)
pred:  tensor([[0.4946],
        [0.4994],
        [0.4930],
        [0.4985],
        [0.5000],
        [0.4909],
        [0.5009],
        [0.5024],
        [0.4986],
        [0.4905],
        [0.4972],
        [0.4936],
        [0.4983],
        [0.4914],
        [0.4998],
        [0.4976]], grad_fn=<SigmoidBackward0>)
loss:  0.24941354990005493
y:  tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)
pred:  tensor([[0.5000],
        [0.4961],
        [0.5029],
        [0.4987],
        [0.5005],
        [0.5007],
        [0.4994],
        [0.4967]